In [3]:

import numpy as np
from numpy import exp
import hw.Henrard as hw     
from hw.const import INPUT_LOG, OUT_LOG
from fox_toolbox.utils import xml_parser as log
from scipy.stats import norm
from numpy import exp, sqrt
from importlib import reload


In [6]:



#### Read IRSM FORM
_, irsmform = log.get_xml(INPUT_LOG)
dsc_curve, sprds = log.get_curves(irsmform)

try:
    estim_curve = sprds[0]
except TypeError:
    estim_curve = dsc_curve

swos = list(log.get_calib_basket(irsmform))
swo = swos[0]

#### END READ IRSM FORM

expiry = swo.expiry
start_date = swo.start_date

fwd = swo.get_swap_rate(dsc_curve, estim_curve)
flt_adjs = swo.get_flt_adjustments(dsc_curve, estim_curve)

#### READ IRSM OUT
_, irsmout = log.get_xml(OUT_LOG)
ref_swo = list(log.get_calib_basket(irsmout))[0]


ref_swos = list(log.get_calib_basket(irsmout))
# ref_swo = ref_swos[0]

ref_mr, (buckets, hw_sigma) = log.get_hw_params(irsmout)

sigma = hw_sigma[0]

#### END READ IRSM OUT



#### MAIN HW

hw_dates = np.insert(swo.payment_dates, 0, start_date)

coef = hw.get_coef(swo.dcfs, swo.strike, flt_adjs)
P_i = hw.get_P_i(dsc_curve, start_date, swo.payment_dates)
alpha_i = hw.get_alpha_i(ref_mr, expiry, sigma, hw_dates)
hw_alphas = hw.get_hw_alphas(sigma, ref_mr, expiry, hw_dates)

x_star = hw.get_x_star(coef, P_i, alpha_i)

price = hw.hw_swo_analytic(coef, P_i, alpha_i, x_star, IsCall=True) * dsc_curve.get_dsc(swo.expiry)

print('fwd recon :', ref_swo.fwd - fwd)
print('Annuity recon', ref_swo.annuity - swo.get_annuity(dsc_curve))
print('x_star', x_star)
print('swap_value', hw.swap_value(coef, P_i, alpha_i, x_star))
print('***Prices***')
print('my HW price', 1000000*price)
print('my numerical HW price', 1000000*hw.hw_swo_numeric(coef, P_i, alpha_i, IsCall=False))
print('out put price', 1000000*ref_swo.tgt_premium ,'with recon', ref_swo.tgt_premium - price )
print(100*(ref_swo.tgt_premium - price)/price, '% difference' )



fwd recon : 7.233536686301889e-14
Annuity recon -6.128431095930864e-14
x_star -1.9948237621088238
swap_value -1.1102230246251565e-16
***Prices***
my HW price 13688.303593235514
my numerical HW price 59.556967868699765
out put price 13706.9401043 with recon 1.8636511064486563e-05
0.13614916514341743 % difference


array([1.01106216e-05, 5.05254192e-05, 1.85773023e-03, 3.56262461e-03,
       5.18893797e-03, 6.73593510e-03])

array([1.01106216e-05, 5.05254192e-05, 1.85773023e-03, 3.56262461e-03,
       5.18893797e-03, 6.73593510e-03])